# Start Here

In the cell below are a few functions that help with plotting data using `matplotlib`. Run the cell to define the functions.

In [ ]:
import os

os.environ["PROJ_LIB"] = "/opt/conda/share/proj"

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def show_plot(x_data, y_data, x_label, y_label):
    """
    Display a simple line plot.
    
    :param x_data: Numpy array containing data for the X axis
    :param y_data: Numpy array containing data for the Y axis
    :param x_label: Label applied to X axis
    :param y_label: Label applied to Y axis
    """
    plt.figure(figsize=(10,5), dpi=100)
    plt.plot(x_data, y_data, 'b-', marker='|', markersize=2.0, mfc='b')
    plt.grid(b=True, which='major', color='k', linestyle='-')
    plt.xlabel(x_label)
    plt.ylabel (y_label)
    plt.show()
    
def plot_box(bbox):
    """
    Display a Green bounding box on an image of the blue marble.
    
    :param bbox: Shapely Polygon that defines the bounding box to display
    """
    min_lon, min_lat, max_lon, max_lat = bbox.bounds
    import matplotlib.pyplot as plt1
    from matplotlib.patches import Polygon
    from mpl_toolkits.basemap import Basemap

    map = Basemap()
    map.bluemarble(scale=0.5)
    poly = Polygon([(min_lon,min_lat),(min_lon,max_lat),(max_lon,max_lat),(max_lon,min_lat)],
                   facecolor=(0,0,0,0.0),edgecolor='green',linewidth=2)
    plt1.gca().add_patch(poly)
    plt1.gcf().set_size_inches(10,15)
    
    plt1.show()
    
def show_plot_two_series(x_data_a, x_data_b, y_data_a, y_data_b, x_label, 
                         y_label_a, y_label_b, series_a_label, series_b_label):
    """
    Display a line plot of two series
    
    :param x_data_a: Numpy array containing data for the Series A X axis
    :param x_data_b: Numpy array containing data for the Series B X axis
    :param y_data_a: Numpy array containing data for the Series A Y axis
    :param y_data_b: Numpy array containing data for the Series B Y axis
    :param x_label: Label applied to X axis
    :param y_label_a: Label applied to Y axis for Series A
    :param y_label_b: Label applied to Y axis for Series B
    :param series_a_label: Name of Series A
    :param series_b_label: Name of Series B
    """
    fig, ax1 = plt.subplots(figsize=(10,5), dpi=100)
    series_a, = ax1.plot(x_data_a, y_data_a, 'b-', marker='|', markersize=2.0, mfc='b', label=series_a_label)
    ax1.set_ylabel(y_label_a, color='b')
    ax1.tick_params('y', colors='b')
    ax1.set_ylim(min(0, *y_data_a), max(y_data_a)+.1*max(y_data_a))
    ax1.set_xlabel(x_label)
    
    ax2 = ax1.twinx()
    series_b, = ax2.plot(x_data_b, y_data_b, 'r-', marker='|', markersize=2.0, mfc='r', label=series_b_label)
    ax2.set_ylabel(y_label_b, color='r')
    ax2.set_ylim(min(0, *y_data_b), max(y_data_b)+.1*max(y_data_b))
    ax2.tick_params('y', colors='r')
    
    plt.grid(b=True, which='major', color='k', linestyle='-')
    plt.legend(handles=(series_a, series_b), bbox_to_anchor=(1.1, 1), loc=2, borderaxespad=0.)
    plt.show()


# Run Time Series and Plot

In the cell below we import the `nexuscli` library and target the NEXUS webservice running in the `nexus-webapp` container.  

Then we define a bounding box and plot it on a map using the `plot_box` function defined above.  

Next, we define the time bounds for our time series and submit the request to NEXUS. The request is timed and the results are then plotted using the `show_plot` method defined above.

In [ ]:
import time
import nexuscli
from datetime import datetime

from shapely.geometry import box

nexuscli.set_target("http://nexus-webapp:8083")

# Create a bounding box using the box method imported above
bbox = box(-150, 40, -120, 55)

# Plot the bounding box using the helper method plot_box
plot_box(bbox)

start = time.perf_counter()

# Call the time_series method for the AVHRR_OI_L4_GHRSST_NCEI dataset using 
# the bounding box and time period 2013-01-01 through 2014-03-01
datasets = ["AVHRR_OI_L4_GHRSST_NCEI"]
start_time = datetime(2015, 11, 1)
end_time = datetime(2015, 11, 30)
ts = nexuscli.time_series(datasets, bbox, start_time, end_time, spark=True)

print("Time Series took {} seconds to generate".format(time.perf_counter() - start))

# Plot the resulting time series using the helper method show_plot
avhrr_ts = ts[0]
show_plot(avhrr_ts.time, avhrr_ts.mean, 'Time', 'Temperature (C)')